In [1]:
#IMPORTS
import json
import os
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime

In [95]:
def get_stats_p1(soup): #Gets basic information such as player name, club, nation, league, age, etc
    info_table = soup.find("table", {"class" : "table table-info"})
    headers = []
    for item in info_table.findAll("th"):
        headers.append(item.text.strip())
    descriptions = []
    count = 0
    for j in info_table.findAll("tr"):
        des = j.find_all("td")[0].text.strip()
        if headers[count].lower() == "accelerate": #if we are looking at the accelerate
            if des[0] == "E":
                des = "Explosive"
            elif des[0] == "C":
                des = "Controlled"
            else:
                des = "Lengthy"
        descriptions.append(des)                 
        count += 1
    return headers, descriptions

def get_stats_p2(soup): #Gets player ratings, stats, overall, and position
    stats = soup.find("div", {"id": "player_stats_json"})
    stats = stats.text.strip()
    stats = json.loads(stats)[0]
    categories = ["Overall", "Position", "Pace", "Passing", "Shooting", "Dribbling", "Defending", "Physical"]
    values = []
    #This block will get a player's overall and position
    values.append((soup.find("div", {"class": "pcdisplay-rat"}).text))
    values.append((soup.find("div", {"class": "pcdisplay-pos"}).text))
    if values[1] == "GK":
        return get_stats_p2_gk(soup)
    for cat in categories:
        if (cat.lower() != "overall" and cat.lower() != "position"):
            values.append(stats[cat.lower()][0]["value"])
    return categories, values

def get_stats_p2_gk(soup): #Gets player ratings, stats, overall, and position
    stats = soup.find("div", {"id": "player_stats_json"})
    stats = stats.text.strip()
    stats = json.loads(stats)[0]
    categories = ["Overall", "Position", "Gkdiving", "Gkhandling", "Gkkicking", "Gkreflexes", "Speed", "Gkpositioning"]
    values = []
    #This block will get a player's overall and position
    values.append((soup.find("div", {"class": "pcdisplay-rat"}).text))
    values.append((soup.find("div", {"class": "pcdisplay-pos"}).text))
    for cat in categories:
        if (cat.lower() != "overall" and cat.lower() != "position"):
            values.append(stats[cat.lower()][0]["value"])
    return categories, values


def get_price_data_daily(player_id): #Gets price data on a player and adds it to a csv (for normal players)
    response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params={
      'api_key': 'ecea4014-8c14-480c-845d-a7ff7ad269ec',
      'url': 'https://www.futbin.com/23/playerGraph?player=' + player_id +'&year=23&type=daily_graph', 
  },
)
    string = response.content.decode("utf-8")
    dic = json.loads(string)
    path = "./Player_Data/" + player_id
    if (os.path.exists(path) == False):
        os.mkdir(path)
    if "ps" in dic:
        data_path = "./Player_Data/" + player_id + "/psdaily_graphFIFA23.csv" #Path to csv data
        lis = dic["ps"]
        time_stamp = []
        price = []
        for item in lis:
            time_stamp.append(item[0])
            price.append(item[1])
        headers = ["Unix Time", "Price"]

        with open(data_path, 'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(headers)
            for i in range(len(price)):
                # write the data
                writer.writerow([time_stamp[i], price[i]])
    if "pc" in dic:
        data_path = "./Player_Data/" + player_id + "/pcdaily_graphFIFA23.csv" #Path to csv data
        lis = dic["pc"]
        time_stamp = []
        price = []
        for item in lis:
            time_stamp.append(item[0])
            price.append(item[1])
        headers = ["Unix Time", "Price"]

        with open(data_path, 'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(headers)
            for i in range(len(price)):
                # write the data
                writer.writerow([time_stamp[i], price[i]])
    else:
        data_path = "./Player_Data/" + player_id + "/nodata.txt"
        with open(data_path, 'w') as fp:
            pass
        
def get_price_data_yday(player_id): #Gets price data on a player and adds it to a csv (for normal players)
    response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params={
      'api_key': 'ecea4014-8c14-480c-845d-a7ff7ad269ec',
      'url': 'https://www.futbin.com/23/playerGraph?player=' + player_id +'&year=23&type=yesterday', 
  },
)
    string = response.content.decode("utf-8")
    dic = json.loads(string)
    path = "./Player_Data/" + player_id
    if (os.path.exists(path) == False):
        os.mkdir(path)
    if "ps" in dic:
        lis = dic["ps"]
        dt = datetime.datetime.utcfromtimestamp(int(lis[0][0])/1000)
        time = (str(dt)[0:10])
        data_path = "./Player_Data/" + player_id + "/" + time + "psydaygraphFIFA23.csv" #Path to csv data
        time_stamp = []
        price = []
        for item in lis:
            time_stamp.append(item[0])
            price.append(item[1])
        headers = ["Unix Time", "Price"]
        with open(data_path, 'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(headers)
            for i in range(len(price)):
                # write the data
                writer.writerow([time_stamp[i], price[i]])
    if "pc" in dic:
        lis = dic["pc"]
        dt = datetime.datetime.utcfromtimestamp(int(lis[0][0])/1000)
        time = (str(dt)[0:10])
        data_path = "./Player_Data/" + player_id + "/" + time + "pcyesterday_graphFIFA23.csv" #Path to csv data
        time_stamp = []
        price = []
        for item in lis:
            time_stamp.append(item[0])
            price.append(item[1])
        headers = ["Unix Time", "Price"]
        with open(data_path, 'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(headers)
            for i in range(len(price)):
                # write the data
                writer.writerow([time_stamp[i], price[i]])
    else:
        data_path = "./Player_Data/" + player_id + "/nodata.txt"
        with open(data_path, 'w') as fp:
            pass
        
                 


def get_player_dic(player_link):#Creates a player_dic
    r = requests.get(
      url='https://proxy.scrapeops.io/v1/',
      params={
          'api_key': 'ecea4014-8c14-480c-845d-a7ff7ad269ec',
          'url': player_link, 
      },
    )
    soup = BeautifulSoup(r.content, 'html.parser')
    results = str(soup.prettify())
    actual_id = results[results.find("data-player-resource") + 22: results.find("data-position") - 2]
    headers = []
    descriptions = []
    h1, d1 = get_stats_p1(soup)
    h2, d2 = get_stats_p2(soup)
    headers = h1 + h2
    descriptions = d1 + d2
    dic = {}
    irrelevant = ["Origin", "Added on", "Club ID", "League ID", "B.Type", "R.Face"]
    for i in range(len(headers)):
        if headers[i] == "Height":
            dic[headers[i]] = descriptions[i][:3]
        elif headers[i] == "Age":
            dic[headers[i]] = descriptions[i][:2]
        elif headers[i] not in irrelevant:
            dic[headers[i]] = descriptions[i]
    dic["ID"] = actual_id
    player_id = dic["ID"]
    path = "./Player_Data/" + player_id
    if (os.path.exists(path) == False):
        os.mkdir(path)
    with open("./Player_Data/" + player_id + "/infoFIFA23.txt", 'w') as convert_file:
        convert_file.write(json.dumps(dic))
    return dic


    

In [ ]:
'''
player_links = []
#Format of Link to loop through player_dataset: https://www.futbin.com/players?page=1
for i in range(100):
    r = requests.get(
          url='https://proxy.scrapeops.io/v1/',
          params={
              'api_key': 'ecea4014-8c14-480c-845d-a7ff7ad269ec',
              'url': "https://www.futbin.com/players?page=" + str(i+1), 
          },
        )
    soup = BeautifulSoup(r.content, 'html.parser')
    player_lis = (soup.findAll("tr", {"class": "player_tr_2"}))
    for item in player_lis:
        item = str(item)
        string = "https://www.futbin.com" + item[item.find("data") + 10: item.find(">") - 1]
        player_links.append(string)
    player_lis = (soup.findAll("tr", {"class": "player_tr_1"}))
    for item in player_lis:
        item = str(item)
        string = "https://www.futbin.com" + item[item.find("data")  + 10: item.find(">") - 1]
        player_links.append(string)
    if (i % 5 == 0):
        print(player_links[-10:])
with open("playerlinks.txt", 'w') as convert_file:
    convert_file.write(json.dumps(player_links))
'''

In [ ]:
#We are on 18 for i
for i in range(106, len(player_links)):
    link = player_links[i]
    dic = get_player_dic(link)
    player_id = dic["ID"]
    player_dic[player_id] = dic
    get_price_data_daily(player_id)
    get_price_data_yday(player_id)
    print(i)
    
    
    

106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


In [67]:
player_links[18]

'https://www.futbin.com/23/player/174/lev-yashin'

In [ ]:
#This block of code gets the player's price data
#Format of Code to get Prices
#Hourly Graph (today): https://www.futbin.com/23/playerGraph?type=today&year=23&player=173731
#Hourly Graph (yday): https://www.futbin.com/23/playerGraph?type=yesterday&year=23&player=173731
#Daily Graph: https://www.futbin.com/23/playerGraph?player=173731&year=23&type=daily_graph


In [ ]:

        writer.writerow([time_stamp[i], price[i]])

In [ ]:
df = pd.read_csv(data_path, sep=",")
# shows top 10 rows
df.head(10)

In [ ]:
#So now we have price data, what's the next plan:
'''
When in day should I buy a player
'''

In [70]:
r = requests.get(
      url='https://proxy.scrapeops.io/v1/',
      params={
          'api_key': 'ecea4014-8c14-480c-845d-a7ff7ad269ec',
          'url': "https://www.futbin.com/23/player/174/lev-yashin", 
      },
    )
soup = BeautifulSoup(r.content, 'html.parser')
stats = soup.find("div", {"id": "player_stats_json"})
stats = stats.text.strip()
stats = json.loads(stats)[0]

{'gkdiving': [{'shortcut': 'DIV', 'gk_shortcut': 'DIV', 'chem_change': 1, 'id': 'gkdiving', 'stat_num': 'igs4', 'name': 'Diving', 'gk_stat_name': 'Diving', 'type': 'main', 'value': 95, 'max': True}, {'shortcut': 'DIV', 'chem_change': 1, 'id': 'gkdiving', 'stat_num': 'igs5', 'name': 'Diving', 'type': 'sub', 'value': 95, 'max': True}], 'gkhandling': [{'shortcut': 'HAN', 'gk_shortcut': 'HAN', 'chem_change': 1, 'id': 'gkhandling', 'stat_num': 'igs11', 'name': 'Handling', 'gk_stat_name': 'Handling', 'type': 'main', 'value': 89, 'max': True}, {'shortcut': 'HAN', 'chem_change': 1, 'id': 'gkhandling', 'stat_num': 'igs12', 'name': 'Handling', 'type': 'sub', 'value': 89, 'max': True}], 'gkkicking': [{'shortcut': 'KIC', 'gk_shortcut': 'KIC', 'chem_change': 1, 'id': 'gkkicking', 'stat_num': 'igs18', 'name': 'Kicking', 'gk_stat_name': 'Kicking', 'type': 'main', 'value': 75, 'max': True}, {'shortcut': 'KIC', 'chem_change': 1, 'id': 'gkkicking', 'stat_num': 'igs19', 'name': 'Kicking', 'type': 'sub', 

In [71]:
stats = soup.find("div", {"id": "player_stats_json"})
stats = stats.text.strip()
stats = json.loads(stats)[0]
categories = ["Gkdiving", "Gkhandling", "Gkkicking", "Gkreflexes", "Speed", "Gkpositioning"]
values = []
for cat in categories:
    values.append(stats[cat.lower()][0]["value"])

categories.append("Overall")
categories.append("Position")
#This block will get a player's overall and position
values.append((soup.find("div", {"class": "pcdisplay-rat"}).text))
values.append((soup.find("div", {"class": "pcdisplay-pos"}).text))
return categories, values

[95, 89, 75, 96, 60, 95, '94', 'GK']


In [91]:
print(get_player_dic("https://www.futbin.com/23/player/174/lev-yashin"))

['Name', 'AcceleRATE', 'Club', 'Nation', 'League', 'Skills', 'Weak Foot', 'Intl. Rep', 'Foot', 'Height', 'Weight', 'Revision', 'Att. WR', 'Def. WR', 'Added on', 'Origin', 'R.Face', 'B.Type', 'DOB', 'ID', 'Club ID', 'League ID'] ['Lev Yashin', 'Controlled', 'FUT ICONS', 'Russia', 'Icons', '1', '3', '4', 'Right', '189cm | 6\'2"', '82', 'Icon', 'Med', 'Med', '2021-08-31', 'Prime', '', 'High & Average', '22-10-1929', '238380', '112658', '2118']
{'Name': 'Lev Yashin', 'AcceleRATE': 'Controlled', 'Club': 'FUT ICONS', 'Nation': 'Russia', 'League': 'Icons', 'Skills': '1', 'Weak Foot': '3', 'Intl. Rep': '4', 'Foot': 'Right', 'Height': '189', 'Weight': '82', 'Revision': 'Icon', 'Att. WR': 'Med', 'Def. WR': 'Med', 'DOB': '22-10-1929', 'ID': '238380', 'Gkdiving': '94', 'Gkhandling': 'GK', 'Gkkicking': 95, 'Gkreflexes': 89, 'Speed': 75, 'Gkpositioning': 96, 'Overall': 60, 'Position': 95}
